In [8]:
import json
import torch
import json
import os
from LayoutDM import CLDM
from torch.utils.data import Dataset, DataLoader
from diffusers import DDPMScheduler
from PIL import Image
import numpy as np

## Load Model with Seperate Param dict

In [7]:
save_path ='/workspace/joonsm/City_Layout/log_dir/FPN[18]/checkpoints/checkpoint-50/pytorch_model.bin'
state = torch.load(save_path)
model = CLDM(use_temp=True, backbone_name='resnet18')
model.load_state_dict(torch.load(save_path), strict=False)
model = model.to(device)
model.eval()

CLDM(
  (extractor): ImageFeatureExtractor(
    (body): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Module(
        (0): Module(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): Module(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d

In [ ]:
a= torch.load(save_path)

In [ ]:
for batch_idx, sample in enumerate(loader):
    print(f"Batch {batch_idx} SR: {sample['sr']}")
    if batch_idx > 0:
        break  # 첫 번째 배치만 확인

In [ ]:
val_path = "/nas2/lait/1000_Members/jjoonvely/CARLA/pre_seg_combined_val.json"

In [ ]:
val = ImageLayout_Val(val_path)
dataset = DataLoader(val, batch_size=len(val))
val_data = next(iter(dataset))

In [ ]:
num_sample  = val_data['box'].shape
model.eval()
noisy_batch = {'image':val_data['image'].to(device),
               'box': torch.rand(*num_sample, dtype=torch.float32, device=device)}
for i in range(1000)[::-1]:
    t = torch.tensor([i]*num_sample[0], device=device)
    with torch.no_grad():
        noise_pred = model(noisy_batch, timesteps=t)
        bbox_pred = diffusion.step(noise_pred, t[0].detach().item(),  noisy_batch['box'], return_dict=True)

        noisy_batch['box'] = bbox_pred.prev_sample


## Check the Area

In [ ]:
predicted = bbox_pred.pred_original_sample
src = val_data['sr']
src_list = []
for i in src:
    src_ = Image.open(i)
    src_list.append(src_)
box = predicted
box = box.cpu().numpy()
box = (box + 1) / 2

match_list = []

for i in range(box.shape[0]):
    img = src_list[i]
    width, height = img.size
    cx,cy,w,h = box[i]
    x = int((cx - w / 2) * width)
    y = int((cy - h / 2) * height)
    x2 = int((cx + w / 2) * width)
    y2 = int((cy + h / 2) * height)
    boxes = (x,y,x2,y2)
    crop = img.crop(boxes)
    crop = np.array(crop)
    blue_channel = crop[:,:,2]
    blue_channel_flatten = blue_channel.flatten()
    match_pixel_size = np.sum(blue_channel_flatten==128)/blue_channel_flatten.size
    match_list.append(match_pixel_size)

value = sum(match_list)/box.shape[0]